In [2]:
import os
from PIL import Image
import glob
import random
from shutil import copyfile
import csv
import pandas as pd
import numpy
from numpy import asarray
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json

In [47]:
def genSageMaker(outputpath,imageoutputpath, files):
    classes = ['3clover','4clover']

    totalcount = 0;
    for f in files:    

        file =f.split('\\')[-1].split('.')[0]
        img_file = f.split('.')[0] + '.jpg'
        img = cv2.imread(img_file)
        img_height, img_width, channels = img.shape

        copyfile(img_file,imageoutputpath+file+'.jpg')
        
        line = {}
        line['file'] = file+'.jpg'
        line['image_size'] = [{
            'width':img_width,
            'height':img_height,
            'depth':3
        }]

        line['annotations'] = []
        line['categories'] = []

        temp_df = pd.read_csv(f, delimiter=' ', header=None, names=['class','x','y','w','h'])

        for i in temp_df.index:
            x = round(temp_df['x'][i] * img_width)
            y = round(temp_df['y'][i] * img_height)
            width = round(temp_df['w'][i] * img_width)
            height = round(temp_df['h'][i] * img_height)
            
            top = y-round(width/2)
            left = x-round(height/2)
            
            
            line['annotations'].append({
                'class_id':int(temp_df['class'][i]),
                'top':int(top),
                'left':int(left),
                'width':int(width),
                'height':int(height)           
            })
            totalcount = totalcount+1

    
        category_df  = temp_df['class'].unique()
        category_df.sort()    
            
        for i in category_df:
            line['categories'].append({
                'class_id':int(i),
                'name':classes[int(i)]
            })


        jsonFile = outputpath + file +".json"
    
        with open(jsonFile,'w') as p:
            json.dump(line,p)
    print(totalcount)

In [48]:
path = "C:\\Users\\kaublezw\\src\\Capstone\\Data\\yolo\\"
files = glob.glob(path+"rotated\\*.txt")

testoutputpath = "C:\\Users\\kaublezw\\src\\Capstone\\Data\\sagemaker\\validation_annotation\\"
trainoutputpath = "C:\\Users\\kaublezw\\src\\Capstone\\Data\\sagemaker\\train_annotation\\"
testimageoutputpath = "C:\\Users\\kaublezw\\src\\Capstone\\Data\\sagemaker\\validation\\"
trainimageoutputpath = "C:\\Users\\kaublezw\\src\\Capstone\\Data\\sagemaker\\train\\"


train, test = train_test_split(files, test_size=.3, random_state=42)

genSageMaker(testoutputpath,testimageoutputpath, test)
genSageMaker(trainoutputpath,trainimageoutputpath, train)



33871
86129
